In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
data = pd.read_json('part-00000-cc1292d9-e874-42fa-aa7d-fa5ecffae4ae-c000.json', lines=True)

In [3]:
van = pd.read_csv('listings.csv.gz')
van = van[['name','latitude','longitude','host_neighbourhood', 'price', 'review_scores_rating']]
van = van.dropna()

In [4]:
data = data[['name','lat','lon']]

In [5]:
X = van[['latitude','longitude']]
y = van[['host_neighbourhood']]
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_valid_scaled = sc.transform(X_valid)

In [6]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_scaled, y_train.values.ravel())
knn.score(X_valid_scaled, y_valid)

0.6797966963151207

In [7]:
bayes = GaussianNB()
bayes.fit(X_train_scaled, y_train.values.ravel())
bayes.score(X_valid_scaled, y_valid)

0.6569250317662008

In [8]:
random_forest = RandomForestClassifier(n_estimators=400, max_depth=7)
random_forest.fit(X_train, y_train.values.ravel())
random_forest.score(X_valid, y_valid)

0.7064803049555273

In [9]:
# reps=5
# dep = np.linspace(1,6,6)
# nodez = [1,3,5,7,10]


# NS = len(nodez)
# M = len(dep)

# rf_oob = np.zeros((M*NS, reps))

# for r in range(reps):
#   counter=0
#   for m in dep:
#     for ns in nodez:
#       wh_rfm = RandomForestClassifier(n_estimators=400, max_depth=m, min_samples_leaf=ns)
#       wh_rfm.fit(X_train,y_train.values.ravel())
#       rf_oob[counter,r] = wh_rfm.score(X_valid,y_valid)
#       counter=counter+1


# parms = expand.grid(nodez,varz)
# row.names(rf.oob) = paste(parms[,2], parms[,1], sep="|")

# mean.oob = apply(rf.oob, 1, mean)
# mean.oob[order(mean.oob)]

# min.oob = apply(rf.oob, 2, min)

# x11(h=7,w=10,pointsize=8)
# boxplot(rf.oob, use.cols=FALSE, las=2)

# x11(h=7,w=10,pointsize=8)
# boxplot(t(rf.oob)/min.oob, use.cols=TRUE, las=2, 
#         main="RF Tuning Variables and Node Sizes")

#np.savetxt("foo.csv", np.round(rf_oob,3), delimiter=",")

In [10]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train.values.ravel())
clf.score(X_valid, y_valid)

0.6416772554002541

In [11]:
model = GradientBoostingClassifier(n_estimators=50,
        max_depth=2, min_samples_leaf=0.1)
model.fit(X_train, y_train.values.ravel())
print(model.score(X_train, y_train.values.ravel()))
print(model.score(X_valid, y_valid))

0.726271186440678
0.6696315120711563


In [12]:
from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors=3)

#standizing the data
from sklearn.preprocessing import MinMaxScaler, StandardScaler
model = make_pipeline(
    StandardScaler(),
    KNeighborsClassifier(n_neighbors=20, weights='distance',  metric = 'euclidean')
)
model.fit(X_train, y_train.values.ravel())
print([model.score(X_train, y_train),model.score(X_valid, y_valid)])

[1.0, 0.6886912325285895]


In [13]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(solver='lbfgs',
    hidden_layer_sizes=(15,15), activation='logistic')
model.fit(X_train_scaled, y_train.values.ravel())

model.score(X_valid_scaled, y_valid)

/home/david1282/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.6848792884371029

In [14]:
###################################
###Predicting the neighbourhood####
###################################

###RANDOM FOREST IS THE BEST#######
random_forest = RandomForestClassifier(n_estimators=400, max_depth=7)
random_forest.fit(X, y.values.ravel())
X_attract = data[['lat','lon']]
attract_neighbour = random_forest.predict(X_attract)

In [15]:
df = pd.DataFrame(attract_neighbour)
df.iloc[:, 0].unique()
data['neighbourhood'] = df.iloc[:, 0]
neighbours_count = data.groupby(['neighbourhood']).count()

neighbours_count['number'] = np.linspace(1,len(neighbours_count),len(neighbours_count), dtype=int)
neighbours_count

,name,lat,lon,number
neighbourhood,,,,
Central Vancouver,3,3,3,1
Chinatown,2,2,2,2
Commercial Drive,1,1,1,3
Downtown Eastside,8,8,8,4
Downtown Vancouver,22,22,22,5
Dunbar-Southlands,4,4,4,6
Fairview,5,5,5,7
Fraserview,4,4,4,8
Grandview-Woodland,2,2,2,9


In [16]:
neighbour_assigned_num = neighbours_count.drop(labels = ['lat', 'lon', 'name'], axis=1)
data = data.join(neighbour_assigned_num, on ='neighbourhood')

In [17]:
import folium
import matplotlib as plt
# define the world map
world_map = folium.Map()

# display world map
world_map

latitude = 49.2823254
longitude = -123.1187994

# Create map and display it
van_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# Display the map of Vancouver
van_map

incidents = folium.map.FeatureGroup()

# add pop-up text to each marker on the map
# latitudes = list(data.lat)
# longitudes = list(data.lon)
# labels = list(data.name)

#creating colors
np.random.seed(101)
# red = np.linspace(0,255,num=len(neighbours_count), dtype = int)
# np.random.shuffle(red)
# blue = np.linspace(0,255,num=len(neighbours_count),  dtype = int)
# np.random.shuffle(blue)
# green = np.linspace(0,255,num=len(neighbours_count), dtype = int)
# np.random.shuffle(green)
rgb = np.array([np.linspace(0,255,num=len(neighbours_count), dtype = int), np.linspace(0,255,num=len(neighbours_count), dtype = int), np.linspace(0,255,num=len(neighbours_count), dtype = int)])
np.random.shuffle(rgb[0])
np.random.shuffle(rgb[1])
np.random.shuffle(rgb[2])
rgb

def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb
#pd.DataFrame(red,green,blue)
def find_colour(point):
    #print(point)
    num = point['number']-1
    return plt.colors.to_hex( rgb[:,num]/255)
    

def placeDot(point):
    folium.Marker(location=[point['lat'], point['lon']],icon = folium.Icon(icon_color=find_colour(point), icon='circle', angle=0, prefix='fa'), popup=point['name'], radius=3).add_to(van_map)

data.apply(placeDot, axis = 1)
# Display the map of Vancouver
van_map
#icon_color=find_colour(point), color="red",icon="hamburger", prefix='fa'


In [18]:
from numpy import cos, arcsin, sqrt, pi, sin


def distance(point, points):
    points_diff = pd.DataFrame()
    points_near = points[points['neighbourhood'] == point['neighbourhood']]
    if len(points_near) > 0: 
        points_diff['lat'] = np.subtract(points_near['lat'], point['lat'] )
        points_diff['lon'] = np.subtract(points_near['lon'], point['lon'] )
        r = 6371
        p = pi/180
        d = 2 * r * arcsin( sqrt( sin(np.array(points_diff['lat'].to_list())*p/2)**2 + cos(point['lat']*p) * cos(points_near['lat']*p) * sin(np.array(points_diff['lon'].to_list())*p/2)**2)) * 1000
        return d 
    return 0
def average_distance(AirBnB,attractions ):
    return np.mean(distance(AirBnB, attractions))


van = van.rename(columns={'latitude': 'lat', 'longitude':'lon', 'host_neighbourhood': 'neighbourhood'})
van['aver_dist'] = van.apply(average_distance, axis = 1, attractions = data)
van

,name,lat,lon,neighbourhood,price,review_scores_rating,aver_dist
0,D1 - Million Dollar View 2 BR,49.288720,-123.120460,Coal Harbour,$150.00,4.64,0.000000
1,Monthly (or Longer ) Designer One Bedroom Down...,49.282010,-123.126690,Downtown Vancouver,$85.00,4.67,950.610097
2,Vancouver's best kept secret,49.256220,-123.066070,Kensington-Cedar Cottage,$140.00,4.91,1476.363299
3,EcoLoft Vancouver,49.249220,-123.081390,Kensington-Cedar Cottage,$150.00,4.75,1627.374612
4,Close to PNE/Hastings Park Garden level suite,49.277210,-123.040860,Hastings-Sunrise,$195.00,4.80,9865.901704
...,...,...,...,...,...,...,...
4244,Delightful 2BR in Downtown+FreePKG+1mintoHarbour,49.279490,-123.108470,Downtown Vancouver,$291.00,5.00,663.016776
4253,Stylish 1BR in Downtown + PARKING,49.279950,-123.105510,Central Vancouver,$208.00,4.75,5501.303387
4258,For 5-Vancouver-PET FRENDLY-2 Bed-room Suite,49.222150,-123.035520,Central Vancouver,$250.00,5.00,13587.657603
4316,Artsy Private Guest Suite with Outdoor Saloon,49.248690,-123.105020,Riley Park–Little Mountain,$89.00,5.00,0.000000


In [19]:
scaler = StandardScaler()

def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

van['price'] = van['price'].apply(clean_currency).astype('float')

aver_dist = np.asarray(van['aver_dist']).reshape(-1,1)
aver_rating = np.asarray(van['review_scores_rating']).reshape(-1,1)
price = np.asarray(van['price']).reshape(-1,1)

scaler.fit(aver_dist)
van['aver_dist_scaled'] = scaler.transform(aver_dist)

scaler.fit(aver_rating)
van['aver_rating_scaled'] = scaler.transform(aver_rating)

scaler.fit(price)
van['price_scaled'] = scaler.transform(price)

In [20]:
van['airbnb_score'] = van['aver_rating_scaled'] - van['aver_dist_scaled'] - van['price_scaled']

In [21]:
max_score_per_neighbourhood = van[['neighbourhood','airbnb_score']].groupby(by = ['neighbourhood']).aggregate('max')
#max_score_per_neighbourhood.set_index('airbnb_score') 
#max_score_per_neighbourhood.index
max_score = van.join(max_score_per_neighbourhood,on='neighbourhood', rsuffix = '_max')
best_airbnbs = max_score[max_score['airbnb_score'] == max_score['airbnb_score_max']]
best_airbnbs

,name,lat,lon,neighbourhood,price,review_scores_rating,aver_dist,aver_dist_scaled,aver_rating_scaled,price_scaled,airbnb_score,airbnb_score_max
62,condos in vancouver BC,49.28052,-123.12691,Central City,339.0,4.00,0.000000,-0.764975,-1.280509,1.148252,-1.663786,-1.663786
733,Yvette's place 2,49.28953,-123.04178,West Hill,45.0,4.57,0.000000,-0.764975,-0.270628,-0.866945,1.361292,1.361292
870,Luxury Condo with Best City Views,49.27814,-123.09953,Chinatown,120.0,4.94,225.629681,-0.707924,0.384909,-0.352864,1.445696,1.445696
1034,Private Bathroom + Bedroom - Amazing Location,49.27737,-123.13023,Burnaby,85.0,5.00,0.000000,-0.764975,0.491212,-0.592768,1.848956,1.848956
1048,Studio+Solarium+Parking,49.27865,-123.12027,Westwood Plateau,90.0,4.82,0.000000,-0.764975,0.172302,-0.558496,1.495774,1.495774
1132,"Very Cozy, Very Home-like, near PNE",49.28483,-123.04634,Hastings-Sunrise,38.0,5.00,10045.236072,1.775021,0.491212,-0.914926,-0.368884,-0.368884
1142,Near Downtown garden private rooms by skytrain,49.25098,-123.11711,Yaletown,85.0,5.00,0.000000,-0.764975,0.491212,-0.592768,1.848956,1.848956
1279,Modern apartment (Mount Pleasant/Olympic Village),49.26978,-123.09867,Waikiki,80.0,5.00,0.000000,-0.764975,0.491212,-0.627040,1.883228,1.883228
1339,Contemporary King Bed w/En Suite On The Park,49.27759,-123.08576,Downtown Eastside,75.0,4.98,1835.868048,-0.300765,0.455777,-0.661313,1.417855,1.417855
1465,Beautiful Room in the Heart of Kitsilano Beach,49.27330,-123.15572,Kitsilano,66.0,5.00,3687.960381,0.167547,0.491212,-0.723002,1.046667,1.046667


In [35]:
latitude = 49.2823254
longitude = -123.1187994

# Create map and display it
van_map_2 = folium.Map(location=[latitude, longitude], zoom_start=12)

incidents = folium.map.FeatureGroup()


def placeDot_2(point):
    folium.Marker(location=[point['lat'], point['lon']],icon = folium.Icon(icon_color='white', icon='hotel', angle=0, prefix='fa'), popup=point['name'], radius=3).add_to(van_map_2)

best_airbnbs.apply(placeDot_2, axis = 1)
# Display the map of Vancouver
van_map_2
#icon_color=find_colour(point), color="red",icon="hamburger", prefix='fa'


In [36]:
attractions = data.drop('number', axis = 1)
best_airbnb_temp = best_airbnbs[['name','lat','lon','neighbourhood']]

all_vertex = pd.concat([best_airbnb_temp,attractions ])

In [23]:
all_vertex

,name,lat,lon,neighbourhood
62,condos in vancouver BC,49.280520,-123.126910,Central City
733,Yvette's place 2,49.289530,-123.041780,West Hill
870,Luxury Condo with Best City Views,49.278140,-123.099530,Chinatown
1034,Private Bathroom + Bedroom - Amazing Location,49.277370,-123.130230,Burnaby
1048,Studio+Solarium+Parking,49.278650,-123.120270,Westwood Plateau
...,...,...,...,...
121,International Food Court,49.285392,-123.117317,Downtown Vancouver
122,Carousel Theatre,49.270443,-123.135278,Fairview
123,Michael J. Fox Theatre,49.216330,-122.984662,Killarney
124,Canada Service,49.263685,-123.133207,Fairview


In [24]:
#This solver is borrowed from https://sandipanweb.wordpress.com/2020/12/08/travelling-salesman-problem-tsp-with-python/
from mip import Model, xsum, minimize, BINARY
from itertools import product
def TSP_ILP(G):
   V1 =  range(len(G))
   n, V = len(G), set(V1)
   model = Model()   # binary variables indicating if arc (i,j) is used 
   # on the route or not
   x = [[model.add_var(var_type=BINARY) for j in V] for i in V]   # continuous variable to prevent subtours: each city will have a
   # different sequential id in the planned route except the 1st one
   y = [model.add_var() for i in V]   # objective function: minimize the distance
   model.objective = minimize(xsum(G[i][j]*x[i][j] \
                               for i in V for j in V))
   
   # constraint : leave each city only once
   for i in V:
      model += xsum(x[i][j] for j in V - {i}) == 1   # constraint : enter each city only once
   for i in V:
      model += xsum(x[j][i] for j in V - {i}) == 1   # subtour elimination
   for (i, j) in product(V - {0}, V - {0}):
      if i != j:
         model += y[i] - (n+1)*x[i][j] >= y[j]-n   # optimizing
   model.optimize()   # checking if a solution was found
   if model.num_solutions:
      print('Total distance {}'.format(model.objective_value))
      nc = 0 # cycle starts from vertex 0
      cycle = [nc]
      while True:
         nc = [i for i in V if x[nc][i].x >= 0.99][0]
         cycle.append(nc)
         if nc == 0:
	         break
	
   return (model.objective_value, cycle)


In [79]:
all_paths = pd.DataFrame(columns = ['neighbourhood', 'path', 'distance'])

from scipy.optimize import linprog

count_index = 0
for i in neighbours_count.index:
    graph_vertex = all_vertex[all_vertex['neighbourhood'] == i]
    #list_path = []

    if len(graph_vertex) > 4 and len(graph_vertex) < 31: #the second condition is pure for the sake of time
        max_int_array = np.full(len(graph_vertex), fill_value=2**31)
        max_int_diagonal = np.diag(max_int_array)
        graph_edge_weight = (graph_vertex.apply(distance, axis = 1, points = graph_vertex) + max_int_diagonal).to_numpy()
        num_columns = len(graph_edge_weight)**2
        num_rows = len(graph_edge_weight)
        path = TSP_ILP(graph_edge_weight)
        path_distance = path[0]
        path_array = path[1]
        path_list = []
        path_neighbourhood = i
        
        graph_vertex_array = graph_vertex.to_numpy()
        for j in range(len(path_array)-1):
            path_list.append(graph_vertex_array[j][0])

        path_obj = {'neighbourhood':path_neighbourhood, 'path':path_list, 'distance':path_distance}
        all_paths = all_paths.append(path_obj, ignore_index = True)


Total distance 14484.745450875616
Total distance 6214.087380236671
Total distance 32604.33688059365
Total distance 3843.61759873198
Total distance 33884.10992279096
Total distance 42368.70609457393
Total distance 10982.208439852744
Total distance 3802.020249934843


In [84]:
all_paths.to_csv('paths')

In [70]:
print(path_obj)

[]


In [71]:
i

5

In [54]:
TSP_ILP(graph_edge_weight)[0]

Total distance 3802.020249934843


3802.020249934843

In [48]:
all_paths[1][1][1]

22

In [52]:
graph_vertex.to_numpy()[0]#[3]for neighbourhood

'West End'

In [26]:
#the below code is borrowed from https://stackoverflow.com/questions/39101137/how-can-i-get-integer-solutions-with-scipy-optimize-linprog, whose author explictedly stated that anyone is free to use or alter the code
# import numpy as np
# from scipy.optimize import linprog
# import copy

# class LP:
#     minimise = True
#     c = None
#     A_ub = None
#     b_ub = None
#     A_eq = None
#     b_eq = None
#     bounds = None
#     method = ""
#     fun = 0.
#     x = None
#     success = False
    
#     def __init__(self,c,minimise=True,Aub=None, bub=None, Aeq=None, beq=None,
#                  bounds=None,method="revised simplex"):
#         self.minimise = minimise
#         if self.minimise:
#             self.c = c
#         else:
#             self.c = -1 * c
#         self.A_ub = Aub
#         self.b_ub = bub
#         self.A_eq = Aeq
#         self.b_eq = beq
#         self.bounds = bounds
#         self.method = method
        
#     def cal(self): 
#         res = linprog(self.c,A_ub=self.A_ub, b_ub=self.b_ub,
#                       A_eq=self.A_eq, b_eq=self.b_eq,bounds=self.bounds,
#                       method=self.method)                      
#         if res["success"] == False:
#             return res["message"]
#         else:
#             self.success = True
        
#             if self.minimise:
#                 self.fun = res["fun"]
#             else:
#                 self.fun = -1 * res["fun"]

#             self.x = res["x"]
            
#     def get_res(self):
#         return (self.x, self.fun, self.success)

# class LP_Data:
    
#     minimise = True
#     c = None
#     A_ub = None
#     b_ub = None
#     A_eq = None
#     b_eq = None
#     bounds = None
#     method = ""
    
#     def __init__(self,c,minimise=True,Aub=None, bub=None, Aeq=None, beq=None,
#                  bounds=None,method="revised simplex"):
#         self.minimise = minimise
#         if self.minimise:
#             self.c = c
#         else:
#             self.c = -1 * c
#         self.A_ub = Aub
#         self.b_ub = bub
#         self.A_eq = Aeq
#         self.b_eq = beq
#         self.bounds = bounds
#         self.method = method

#     def set_bounds(self,bounds_list):
#         self.bounds = bounds_list

# class LPd:
    
#     data = []
#     fun = 0.
#     x = []
#     success = False
#     result = None
    
#     def __init__(self,lp_data):
#         self.data = lp_data

#     def cal(self):
#         result = None
#         res = linprog(self.data.c,A_ub=self.data.A_ub, b_ub=self.data.b_ub,
#                       A_eq=self.data.A_eq, b_eq=self.data.b_eq,
#                       bounds=self.data.bounds,
#                       method=self.data.method)                      
#         if res["success"] == False:
#             self.result = ([], np.NaN, False, None)
#         else:
#             self.success = True
#             if self.data.minimise:
#                 self.fun = res["fun"]
#             else:
#                 self.fun = -1 * res["fun"]
#             self.x = res["x"]
#             self.result = (self.x, self.fun, self.success, self.data)
            
#     def get_res(self):
#         return self.result

# def level_iterator(level0, int_index):                                  

#     level1 = []
#     for k in range(len(level0)):
#         for i in int_index:
#             if level0[k][0][i-1] != np.floor(level0[k][0][i-1]):
#                 level0[k][3].bounds[i-1] = (0,np.floor(level0[k][0][i-1]))
#                 lp = LPd(copy.deepcopy(level0[k][3]))
#                 lp.cal()
#                 output = lp.get_res()
#                 level1.append(output)
#                 level0[k][3].bounds[i-1] = (np.ceil(level0[k][0][i-1]),None)
#                 lp = LPd(copy.deepcopy(level0[k][3]))
#                 lp.cal()
#                 output = lp.get_res()
#                 level1.append(output)
#                 break
#     return level1

# def intsol(solution,int_index):
#     is_int = True 
#     for i in int_index: 
#         if solution[0][i-1] != np.floor(solution[0][i-1]):
#             is_int = False
#             break
#     return is_int
 
# def feasiblelevl(level, solution, int_index):
#     newlevel = []
#     solution = solution
#     for k in range(len(level)):
#         lp = level[k]
#         if len(lp[0]) > 0:
#             if lp[2] == False:
#                 pass
#             elif intsol(lp,int_index) and lp[1] >= solution[1]:
#                 solution = lp
#             elif lp[1] > solution[1]:
#                 newlevel.append(lp)
#     return (newlevel, solution)

# def LPmi(data, int_index):
#     level0 = []
#     lp  = LPd(data)
#     lp.cal()
#     level0.append(lp.get_res())
#     solution = [None,-np.inf,None,None]
#     level0 = (level0, solution)
#     level0 = feasiblelevl(level0[0], solution, int_index)
#     if len(level0[0]) == 0:
#         return level0[1][0:3]
#     else:
#         for k in range(10):
#             level1 = level_iterator(level0[0],int_index)
#             level1 = feasiblelevl(level1, solution, int_index)
#             if len(level1[0]) == 0:
#                 break
#             level0 = level1
#         return level1[1][0:3]

# def test():
#     c = np.array([3.,7])
#     minimise = False
#     A_ub = np.array([[7.,8],[1,3]])
#     b_ub = np.array([56,12])
#     data = LP_Data(c,minimise,A_ub,b_ub,bounds=[(0,None),(0,None)])
#     int_index = [2] #or [1,2] or [] or [1]#
#     out = LPmi(data,int_index)
#     print(out)
# test()

In [27]:
# from scipy.optimize import linprog
# max_int_array = np.full(len(graph_vertex), fill_value=2**31)
# max_int_diagonal = np.diag(max_int_array)
# graph_edge_weight = (graph_vertex.apply(distance, axis = 1, points = graph_vertex) + max_int_diagonal).to_numpy()
# num_columns = len(graph_edge_weight)**2
# num_rows = len(graph_edge_weight)
# all_paths.append(TSP_ILP(graph_edge_weight))

# #creating linear-program
# # graph_edge_weight[num_rows - 1].min

# # start_dist = np.array(graph_edge_weight[num_rows - 1])
# # start_dist = np.reshape(start_dist, newshape= num_rows)
# # end_point = np.where(start_dist == np.amin(start_dist))[0][0]
# # start_point = num_rows-1

# A = np.empty([num_rows*2 ,num_columns + num_rows ])
# A.fill(0)

# for i in range(num_columns):
#     pos_index = int(i/num_rows)
#     neg_index = int(i%num_rows)
#     # if(pos_index != end_point):
#     A[pos_index][i] = 1
#     # if(neg_index != start_point):
#     A[neg_index+num_rows][i] = -1
#     # if(pos_index == neg_index):
#     #     A[neg_index][i] = 0
#     #     A[pos_index][i] = 0


# #A[num_rows] = A[num_rows-1]
# c = np.concatenate((np.reshape(graph_edge_weight, newshape=num_columns), np.zeros(num_rows)))

# b = np.empty(len(A))
# b.fill(-1)
# for j in range(num_rows):
#     b[j] = 1
# # b[end_point] = -1
# # b[start_point] = 1

# minimize = True

# A_ub = np.zeros([num_columns ,num_columns + num_rows ], dtype = int)
# A_ub.fill(0)
# # for count in range(num_columns):
# count = 0
# for i in range(num_rows):
#     for j in range(num_rows):
#         if(j!=i):
#             y_j_index = num_columns+j
#             y_j = 1
#             y_i_index = num_columns+i
#             y_i = -1
#             x_ij_index =num_rows*i + j-1
            
#             x_ij = num_rows+1
#             A_ub[count,y_j_index] = y_j
#             A_ub[count,y_i_index] = y_i
#             A_ub[count,x_ij_index] = x_ij
#         count+=1

# b_ub = np.empty(num_columns)
# b_ub.fill(num_rows)


# lp_data = LP_Data(c,minimise=True,Aub=A_ub, bub=b_ub, Aeq=A, beq=b,
#                 bounds=(0,1),method="revised simplex")
# int_index = [2] #or [1,2] or [] or [1]#
# out = LPmi(lp_data,int_index)
# #print(out[0])
# # solution_matrix = np.reshape(out[0], newshape=(num_rows+1, num_rows))
# # solution_tuples = []#np.empty((num_rows,2))
# # l = num_rows-1
# # m = 0

# # # if(solution_matrix!=None):
# # for k in range(num_rows):
# #     for m in range(num_rows):
# #         if solution_matrix[l][m] == 1:
# #             solution_tuples = np.append(solution_tuples, [l,m])
# #             print(l,m)
# #             l = m
# #             k += 1


# #    c,minimise,A_ub,b_ub,bounds=[(0,None),(0,None)])
# #linprog(c, A_ub=None, b_ub=None, A_eq=A, b_eq=b, bounds=(0,1), method='interior-point', callback=None, options=None, x0=None)

In [28]:
# attractions = data
# AirBnB = van.iloc[0]
# points_diff = pd.DataFrame()
# points_diff['lat'] = np.subtract(attractions['lat'], AirBnB['latitude'] )
# points_diff['lon'] = np.subtract(attractions['lon'], AirBnB['longitude'] )
# r = 6371
# p = pi/180
# d = 2 * r * arcsin( sqrt( sin(np.array(points_diff['lat'].to_list())*p/2)**2 + cos(AirBnB['lat']*p) * cos(attractions['lat']*p) * sin(np.array(points_diff['lon'].to_list())*p/2)**2)) * 1000
# d